# Test of LINE Notify Bot

## Libraries

In [7]:
import requests

## Setting

Access token can be obtained from [LINE Notify](https://notify-bot.line.me/ja/).

In [3]:
access_token = '7mWudLiqSmALASm8sXN0cJgHRD8Aqz2fuz7gQ4qQMM6'

# when access token is written in access_token.txt
with  open('access_token.txt', 'r') as fin:
    access_token = fin.read().strip()

FileNotFoundError: [Errno 2] No such file or directory: 'access_token.txt'

In [3]:
headers = {'Authorization': 'Bearer ' + access_token}

In [4]:
url = "https://notify-api.line.me/api/notify"

## Send Message

In [5]:
message = 'only message'

In [6]:
payload = {'message': message}
requests.post(url, headers=headers, params=payload)

<Response [200]>

## Send Image

In [7]:
message = 'with image'
image = 'test.png'  # png or jpg

In [8]:
payload = {'message': message}
files = {'imageFile': open(image, 'rb')}
requests.post(url, headers=headers, params=payload, files=files)

<Response [200]>

if message is None, image is not sent and no error occurs (but the status code of the response is 400).

In [9]:
payload = {'message': ''}
requests.post(url, headers=headers, params=payload, files=files)

<Response [400]>

## Send Sticker

Sticker and its package IDs are chosen from: https://devdocs.line.me/files/sticker_list.pdf

In [10]:
payload = {
    'message': 'with sticker',
    'stickerPackageId': 1,
    'stickerId': 13,
    }
requests.post(url, headers=headers, params=payload)

<Response [200]>

if message is None, sticker is not sent and no error occurs (but the status code of the response is 400).

In [11]:
payload = {
    'message': '',
    'stickerPackageId': 1,
    'stickerId': 13,
    }
requests.post(url, headers=headers, params=payload, files=files)

<Response [400]>

if sticker and its package IDs do not exist, message is not sent and no error occurs

In [12]:
payload = {
    'message': 'with sticker',
    'stickerPackageId': 1,
    'stickerId': 10000,
    }
requests.post(url, headers=headers, params=payload, files=files)

<Response [400]>

## Test of LINENotifyBot Module

### From Command Line

In [13]:
!python ../line_notify_bot.py access_token.txt "test from command line" -i test.png -sp 1 -s 13

### As Module

In [1]:
import sys; sys.path.append('../')
from line_notify_bot import LINENotifyBot

In [2]:
bot = LINENotifyBot(access_token=access_token)

NameError: name 'access_token' is not defined

In [16]:
bot.send(
    message='test of module',
    image='test.png',  # png or jpg
    sticker_package_id=1,
    sticker_id=13,
    )

<Response [200]>

### Errors

if message is None

In [17]:
bot.send(message='')

ValueError: message must not be None.

if message is not str

In [18]:
bot.send(message=1)

TypeError: message must be str. But input message type was: <class 'int'>

if image file does not exist

In [19]:
bot.send(
    message='test of image file error',
    image='images/test.png',
    )

ValueError: image file does not exist: images\test.png

if sticker ID is input but package ID is not input

In [20]:
bot.send(
    message='test of sticker error',
    sticker_id=1,
    )

ValueError: sticker_id is input, but sticker_package_id is not input.

if wrong sticker ID is input

In [21]:
bot.send(
    message='test of sticker error',
    sticker_package_id=1,
    sticker_id=10000,
    )

Exception: The message, image, and sticker can not be sent for some reason. Maybe you input wrong sticker (or sticker package) ID.

In [4]:
class LINENotifyBot:
    """Sends message, image, and sticker to a LINE group.
    Access token can be obtrained from: https://notify-bot.line.me/ja/

    Args:
        access_token (str):
            Access token necessary to tap LINE Notify API.

    Attributes:
        __headers (dict): Necessary to send request to LINE Notify.
        API_URL (str): URL of LINE Notify API.
    """
    API_URL = 'https://notify-api.line.me/api/notify'
    def __init__(self, access_token):
        self.__headers = {'Authorization': 'Bearer ' + access_token}

    def send(self, message, image=None, sticker_package_id=None, sticker_id=None):
        """Sends message, image, and sticker to a LINE group designated by access token.
        Sticker and its package IDs can be chosen from: https://devdocs.line.me/files/sticker_list.pdf

        Args:
            message (str): Sent message. Maximum length is 1000.
                If is None, message can't be sent.
                In this situation, image and sticker also can't be sent.
            image (str or pathlib.Path): Path to image to send.
                Available extentions are only png or jpeg.
                There is a limitation to the number of images
                which can be sent in one hour (1,000 images by default).
                Defaults to None.
            sticker_package_id (int): ID of sticker package.
                If set to None, only message is sent.
                If set to unregistered integer,
                even message can't be sent.
                Defaults to None.
            sticker_id (int): ID of sticker.
                If set to None, only message is sent.
                If set to unregistered integer,
                even message can't be sent.
                Defaults to None.

        Returns:
            response (requests.models.Response): Response of the requests.post.
        """
        if not message:
            raise ValueError('message must not be None.')
        if not isinstance(message, str):
            raise TypeError(f'message must be str. But input message type was: {type(message)}')

        if sticker_package_id:
            if isinstance(sticker_package_id, int):
                pass
            elif isinstance(sticker_package_id, str):
                if sticker_package_id.isdecimal():
                    sticker_package_id = int(sticker_package_id)
                else:
                    raise ValueError(f'sticker_package_id can be an arabic decimal str, but the input was: {sticker_package_id}.')
            else:
                raise TypeError('sticker_package_id must be int or arabic decimal str, but the input type was: {type(sticker_package_id)}')

            if not sticker_id:
                raise ValueError('sticker_package_id is input, but sticker_id is not input.')

        if sticker_id:
            if isinstance(sticker_id, int):
                pass
            elif isinstance(sticker_id, str):
                if sticker_id.isdecimal():
                    sticker_id = int(sticker_id)
                else:
                    raise ValueError(f'sticker_id can be an arabic decimal str, but the input was: {sticker_id}.')
            else:
                raise TypeError('sticker_id must be int or arabic decimal str, but the input type was: {type(sticker_id)}')

            if not sticker_package_id:
                raise ValueError('sticker_id is input, but sticker_package_id is not input.')


        payload = {
            'message': message,
            'stickerPackageId': sticker_package_id,
            'stickerId': sticker_id,
            }
        files = {}

        if image:
            if isinstance(image, str):
                image_path = Path(image)
            elif isinstance(image, Path):
                image_path = image
            else:
                raise TypeError(f'image must be str or pathlib.Path. But input image type was: {type(image)}')

            if not image_path.exists():
                raise ValueError(f'image file does not exist: {image_path}')

            extention = image_path.suffix
            if not extention:
                raise ValueError('image must not be a directory. But the input was: {image_path}')
            elif extention not in ['.png', '.jpg']:
                raise ValueError(f'image must have .png or .jpg as its extention. But {extention} file was input.')

            files = {'imageFile': open(str(image_path), 'rb')}

        response = requests.post(
            LINENotifyBot.API_URL,
            headers=self.__headers,
            data=payload,
            files=files,
            )

        if not response.ok:
            raise Exception('The message, image, and sticker can not be sent for some reason. Maybe you input wrong sticker (or sticker package) ID.')

        return response

In [9]:
bot = LINENotifyBot(access_token='7mWudLiqSmALASm8sXN0cJgHRD8Aqz2fuz7gQ4qQMM6')

bot.send(
    message='Write Your Message!'
    )

<Response [200]>